In [ ]:
from mongoengine import *
import datetime

In [ ]:
connect('AI')

#### User class

In [ ]:
class User(Document):
    email = StringField(required=True, primary_key=True)
    first_name = StringField(max_length=50)
    last_name = StringField(max_length=50)
    create_date = DateTimeField(default=datetime.datetime.now)
    age = IntField(default=30)

#### define documents

In [ ]:
class Comment(EmbeddedDocument):
    content = StringField()
    name = StringField(max_length=120)

In [ ]:
class Post(Document):
    meta = {'allow_inheritance': True}    # This is critical for inheritance to work
    
    title = StringField(max_length=120, required=True)
    author = ReferenceField(User)
    tags = ListField(StringField(max_length=30))
    comments = ListField(EmbeddedDocumentField(Comment))

In [ ]:
class TextPost(Post):
    content = StringField()

class ImagePost(Post):
    image_path = StringField()

class LinkPost(Post):
    link_url = StringField()

#### create users

In [ ]:
ross = User(email='ross@example.com', first_name='Ross', last_name='Lawley', age=30).save()
john = User(email='john@example.com', first_name='John', last_name='Sam', age=25).save()
tom = User(email='tom@example.com', first_name='tom', last_name='johnny', age=100).save()

### John changes his name

In [ ]:
john.name = 'johnny'
john.save()

John make comments

In [ ]:
comment1 = Comment(content='this is a great post', name='john')
comment2 = Comment(content='Mongo DB is a great database', name='john')

create posts

In [ ]:
post1 = TextPost(title='Fun with MongoEngine', author=john)
post1.content = 'Took a look at MongoEngine today, looks pretty cool.'
post1.tags = ['mongodb', 'mongoengine']
# Add comment to post
post1.comments.append(comment1)
post1.save()

post2 = LinkPost(title='MongoEngine Documentation', author=ross)
post2.link_url = 'http://docs.mongoengine.com/'
# Add comment to post
post2.comments.append(comment2)
post2.tags = ['mongoengine']
post2.save()

Access documents to print all comments

In [ ]:
for post in Post.objects:
    for comment in post.comments:
        print("{} made comment with following {}".format(comment.name, comment.content))

### Dynamic Documents

In [ ]:
class TestData(DynamicDocument):
    test_case = StringField(max_length=200, required=True)

In [ ]:
data = {'setup':'eagle',
       'throughput': 123,
       'vals': [1, 2, 3],
       }

In [ ]:
test1 = TestData(test_case='RF test data 1')
for field, val in data.items():
    test1[field] = val
test1.save()

### print first name of users with age <= 30, join field name and operator with double underscore

In [ ]:
for user in User.objects(age__lte=30):
    print(user.first_name)

In [ ]:
import string
def generate_names(name_len=3):
    firstnames = []
    def combos(path):
        nonlocal firstnames
        if len(path) == name_len:
            firstnames.append(''.join(path))
            return
        for ii in string.ascii_lowercase:
            combos(path+[ii])
    combos([])
    return firstnames

In [ ]:
names=generate_names()

In [ ]:
len(names)==26*26*26

In [ ]:
def generate_fake_users(name_len=3):
    firstnames = generate_names(name_len=name_len)
    lastnames = (name[::-1] for name in firstnames)
    emails = (firstname+'_'+lastname+"@gmail.com" 
              for firstname, lastname in zip(firstnames, lastnames))
    return (User(email=email, first_name=firstname, last_name=lastname)
           for email, firstname, lastname in zip(emails, firstnames, lastnames))

In [ ]:
fake_users = generate_fake_users(4)

In [ ]:
for user in fake_users:
    user.save()

In [ ]:
t0 = datetime.datetime.now()
res = []
for user in User.objects(email='aaa_bbb@gmail.com'):
    res.append(user.lastname)
dt = datetime.datetime.now() - t0

### Atomic updates
http://docs.mongoengine.org/guide/querying.html#atomic-updates


Atomicity: https://en.wikipedia.org/wiki/Atomicity_(database_systems)

In [ ]:
class data_structure(Document):
    counter = IntField(default=0)
    data = ListField(IntField)

#### create a document

In [ ]:
ds = data_structure()
ds.save()

#### atomic updates 

In [ ]:
# update_one: Perform an atomic update on the fields of the first document matched by the query.
data_structure.objects(id=ds.id).update_one(inc__counter=10)

In [ ]:
data_structure.objects(id=ds.id).update_one(push__data=666)

In [ ]:
data_structure.objects(id=ds.id).update_one(upsert=True, **{'push_all__data':[100, 200, 300]})